# RNN

## Overview

When humans think, we are thinking based on the understanding of previous time but not from scratch. Traditional neural networks can’t do this, and it seems like a major shortcoming. They can't "remember" what they processed 2 seconds ago. For example, imagine you want to do sentimental analysis of some texts. It will be unclear if the traditional network cannot recognize the short phrase and sentences, or understand sequential data in any meaningful way.

Recurrent neural networks address this issue. They are networks with loops in them, allowing information to persist.

<img src="https://i.imgur.com/9WQtoov.png" width="500"/>

A recurrent neural network can be thought of as multiple copies of the same network, each passing a message to a successor. Consider what happens if we unroll the above loop:
 
<img src="https://i.imgur.com/jcMYgPG.png" width="500"/>

Recurrent neural networks may be connected in many different ways: sequences in the input, the output, or in the most general case both.

<img src="https://i.imgur.com/gcROLRW.png" width="700"/>

## Implementation

In our case, we implemented rnn with modules offered by the package of `keras`. To use `keras` and our module, you must have both `tensorflow` and `keras` installed as a prerequisite. `keras` offered very well defined high-level neural networks API, enabling you to build both convolutional neural networks and recurrent neural networks with simple syntax in Python!

To view our implementation of a simple rnn, please use the following code:

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN
from keras.layers import LSTM
import pandas as pd
import matplotlib.pyplot as plt

2023-03-23 07:47:36.128052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 07:47:36.520045: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-23 07:47:36.520091: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-23 07:47:37.802368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
### Load the data
num_words=5000
data = imdb.load_data(num_words=num_words)

### Train / Test splitting
(X_train, y_train), (X_test, y_test) = data

The IMDb dataset is a collection of movie reviews from the Internet Movie Database (IMDb) website, which is a popular platform for information related to movies, TV shows, and other video content. This dataset is commonly used for sentiment analysis tasks in natural language processing (NLP) and machine learning.

The dataset consists of 50,000 movie reviews, each labeled as either positive (1) or negative (0) based on the sentiment expressed in the review. The dataset is divided into two sets: 25,000 reviews for training and 25,000 reviews for testing. This allows us to train a model on one set of data and evaluate its performance on another, unseen set of data.

The movie reviews in the dataset have been preprocessed, and each review is represented as a list of integers. Each integer corresponds to a specific word in the review. The words have been indexed by frequency in the entire dataset, with the most common word having an index of 1, the second most common word having an index of 2, and so on. For example, the integer 4 would represent the 4th most common word in the dataset.

In this preprocessed form, the dataset is ready to be used in various NLP tasks, such as sentiment analysis using machine learning models, including recurrent neural networks (RNNs).

In [ ]:
for review in X_train[:3]:
    print(f"Example Review: {review}")
    print("---------")

In [ ]:
# Analyze review lengths
review_lengths = [len(review) for review in X_train]

# Create a Pandas DataFrame
review_data = pd.DataFrame({"review": X_train, "length": review_lengths, "sentiment": y_train})

# Display basic statistics
print("Basic Statistics:")
print(review_data.describe())

# Plot the distribution of review lengths
plt.figure(figsize=(10, 5))
plt.hist(review_lengths, bins=50, edgecolor='black')
plt.title("Review Length Distribution")
plt.xlabel("Review Length")
plt.ylabel("Frequency")
plt.show()

# Analyze the most common words
word_index = imdb.get_word_index()
index_to_word = {index: word for word, index in word_index.items()}

word_counts = {}

for review in X_train:
    for word_index in review:
        word = index_to_word.get(word_index, "")
        if word not in word_counts:
            word_counts[word] = 1
        else:
            word_counts[word] += 1

# Top 10 most common words
most_common_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:10]
print("\nTop 10 Most Common Words:")
for word, count in most_common_words:
    print(f"{word}: {count}")

The reviews have different lengths, so we need to pad them to ensure consistent input sizes for our RNN. 

Padding is a technique used in natural language processing and sequence data processing to ensure that all input samples have the same length. In the context of the IMDb dataset, movie reviews have varying lengths, which can cause issues when feeding them into machine learning models, as most models require fixed-size input.

In the given code snippet, the sequence.pad_sequences() function from Keras is used to pad the movie reviews to a consistent length:

In [ ]:
def pad_sequences(sequences, maxlen):
    padded_sequences = []
    
    for seq in sequences:
        if len(seq) < maxlen:
            # Pad the sequence with 0s at the beginning
            padded_seq = [0] * (maxlen - len(seq)) + seq
        else:
            # Truncate the sequence to maxlen
            padded_seq = seq[:maxlen]
        
        padded_sequences.append(padded_seq)
    
    return padded_sequences

max_review_length = 500
X_train = np.array(pad_sequences(X_train, maxlen=max_review_length))
X_test = np.array(pad_sequences(X_test, maxlen=max_review_length))

Now, let's create our RNN model using Keras' Sequential model. We'll start with an Embedding layer, followed by a SimpleRNN layer, and finally a Dense layer for the output.

In the context of building a Recurrent Neural Network (RNN) using Keras, the Sequential model is a linear stack of layers that we can easily create by passing a list of layer instances. In this case, we're using three types of layers: Embedding, SimpleRNN, and Dense. Let's break down each layer and how they contribute to the model:

* **Embedding layer**: The Embedding layer is used to convert the input integers (word indices) into dense vectors of fixed size. These dense vectors can be thought of as a continuous representation of words in a high-dimensional space. The Embedding layer essentially creates a lookup table that maps each word index to a dense vector. This layer is crucial for feeding the text data into the RNN, as it allows the model to learn meaningful word representations and better capture the relationships between words.

* **SimpleRNN layer**: This is the main building block of our Recurrent Neural Network. The SimpleRNN layer implements a simple RNN architecture, which has internal loops that allow it to maintain a hidden state across time steps. This hidden state captures information about the previous time steps and helps the model learn patterns across sequences of data. In our case, the sequences are movie reviews, and the SimpleRNN layer helps the model learn the relationships between words in a review to predict its sentiment.

* **Dense layer**: The Dense layer, also known as a fully connected layer, is used for the output of our RNN model. This layer takes the output from the SimpleRNN layer and maps it to a single output value (in this case, the sentiment of the review). The activation function used in this layer is 'sigmoid', which maps the output value to a range between 0 and 1. This allows us to interpret the output as a probability representing the sentiment of the review (positive or negative).

In [ ]:
embedding_vector_length = 32

model = Sequential()
model.add(Embedding(5000, embedding_vector_length, input_length=max_review_length))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

# from keras.utils import plot_model
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In the process of building a neural network model, the following two steps are crucial before training the model on the data:

1. **Compiling the model**: Compiling the model means configuring the model for training by defining the loss function, the optimizer, and the evaluation metric(s). The loss function calculates the difference between the predicted output and the actual output (ground truth). The optimizer is responsible for updating the weights of the network to minimize the loss function.

2. **Training the model**: Training the model means feeding the training data (input features and corresponding labels) into the model, so it can learn to make accurate predictions. The model adjusts its weights iteratively through a process called backpropagation to minimize the loss function. The training process typically involves multiple epochs (iterations through the entire dataset) and can also include splitting the data into mini-batches for more efficient learning.

In [ ]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)


Here, we're using the binary_crossentropy loss function because we have a binary classification problem (positive or negative sentiment). The adam optimizer is a popular choice for training neural networks because it adapts the learning rate during training, which can lead to faster convergence. The accuracy metric is used to evaluate the performance of the model during training.

After compiling the model, you can train it using the `fit` method:

In [ ]:
model.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    epochs=3, 
    batch_size=64
)


Now, we can interactively visualize its predictions and start to explore our model... This involves:

1. Define a function to decode the input sequences back into text.
2. Create a function to preprocess new input data (reviews) for the model.
3. Test the model by providing new reviews and visualize the predicted sentiment.

In [ ]:
# Step 1: Define a function to decode sequences back into text
def decode_review(encoded_review):
    index_to_word = {value: key for key, value in imdb.get_word_index().items()}
    decoded_review = ' '.join([index_to_word.get(index - 3, '?') for index in encoded_review])
    return decoded_review

# Decode and display the first 5 reviews from the testing dataset
for i in range(5):
    decoded_review = decode_review(X_test[i])
    print(f"Review {i + 1}:\n{decoded_review}\n")

We can see that all of our reviews are padded at the beginning to ensure they are the same length. Note that the decoded reviews may contain some special tokens like ?, which represent out-of-vocabulary words or words that were replaced during the preprocessing step when we loaded the IMDb dataset. Now, lets see how accurate our RNN is at predicting review sentiment!

In [ ]:
# Evaluate the model on the testing data
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Not bad!! Want to try it yourself?

In [ ]:
### Provide a review here!!
MY_REVIEW = "What a captivating film!"

In [ ]:
# Step 2: Create a function to preprocess YOUR review
def preprocess_review(text_review):
    # Tokenize the review
    words = text_review.split()
    word_index = imdb.get_word_index()
    
    # Encode the review using the IMDb word index
    encoded_review = [word_index.get(word, 2) if word_index.get(word, 2) < 5000 else 2 for word in words]
    
    # Pad the encoded review
    padded_review = pad_sequences([encoded_review], maxlen=max_review_length)
    
    return padded_review

# Step 3: Test the model by providing new reviews
preprocessed_review = preprocess_review(MY_REVIEW)

prediction = model.predict(preprocessed_review)
sentiment = "positive" if prediction[0][0] > 0.5 else "negative"

print(f"Review: {MY_REVIEW}")
print(f"Sentiment: {sentiment} (probability: {prediction[0][0]:.2f})")

# LSTM

In a standard RNN, the gradient flow is faulty. As a consequence, the RNN did not learn a longer context. With LSTMs, we want to counter the problem. The architecture of Long Short-Term Memory Networks provides a better way for the gradients to backpropagate.

A standard RNN faces a problem called the "*vanishing gradient*," which makes it difficult for the network to learn long-range dependencies in sequences. This means that when training an RNN, it struggles to connect information from earlier parts of the sequence to information later in the sequence, limiting its ability to learn complex patterns.

LSTMs were designed to address this issue. They have a unique architecture that allows them to "remember" information over longer periods, making it easier to learn the relationships between earlier and later parts of a sequence. This is achieved through the use of gates, which control the flow of information in the LSTM cell. The gates selectively update the internal memory of the LSTM, allowing it to retain important information while discarding irrelevant details.

The LSTM architecture helps maintain the gradient flow during backpropagation, which is the process of updating the network's weights based on the error it made during training. By preserving the gradient flow, LSTMs can learn long-range dependencies more effectively than standard RNNs, making them better suited for tasks involving sequences with varying lengths and complex patterns.

<img src="https://i.imgur.com/IFmjkLS.png" width="500"/>

This moving left to right, we see three gates:

1. **Forget gate**: This gate decides which information should be removed from the cell state. Like the input gate, it uses a sigmoid activation function to output values between 0 and 1 for each element in the cell state. A value close to 1 means the information should be retained, while a value close to 0 means the information is no longer relevant and can be forgotten.

2. **Input gate**: This gate determines which new information should be added to the cell state. It consists of a sigmoid activation function that outputs values between 0 and 1, indicating the importance of each input element. A value close to 1 means the input is significant and should be stored, while a value close to 0 means the input is not important and can be discarded.

3. **Output gate**: This gate controls what information from the cell state is used to compute the final output of the LSTM cell at the current time step. The output gate also uses a sigmoid activation function to produce values between 0 and 1, which are then multiplied element-wise with the cell state (after passing through a tanh activation function) to generate the final output.

The gating mechanism enables LSTMs to selectively update their cell state and output, which allows them to learn and maintain long-range dependencies more effectively than traditional RNNs. This is particularly useful in tasks where the input sequences have variable lengths and important information can be located at different positions in the sequence. This selective updating of the memory state helps the LSTM to maintain important context over long sequences, making it a more powerful and flexible model for sequence-to-sequence tasks compared to standard RNNs, which struggle with the vanishing gradient problem and have difficulty capturing long-range dependencies.

Thinking of how these gates coordinate with one another, we can see why LSTMs are preferred over RNNs.

Upon looking at the diagram below, we notice that the gradients within this memory state flow without much perturbation. The information flows freely in the circuit provided for the memory state, rather than being lost to a vanishing gradient in the case of RNNs.

<img src="https://i.imgur.com/xO6IV0X.png" width="500"/>

The model is able to understand far more complex relationship in distantly-related datapoints than was possible with RNNs.

Let's see how much better this does than our RNN... We simply need to refactor our code to call the LSTM model with `keras`!


In [ ]:
# Define the LSTM model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(num_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=3,
    batch_size=64
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Let's see how much better it does with yours!

In [ ]:
MY_REVIEW = MY_REVIEW # you set this earlier
preprocessed_review = preprocess_review(MY_REVIEW)
prediction = model.predict(preprocessed_review)
sentiment = "positive" if prediction[0][0] > 0.5 else "negative"

print(f"Review: {MY_REVIEW}")
print(f"Sentiment: {sentiment} (probability: {prediction[0][0]:.2f})")